# CMSC320: Checkpoint 2
Tyshon Brown, David Li, Zile Liu, and Samuel Opoku-Agyemang


## Data Pre-Processing

### Imports

In [13]:
import pandas as pd
import numpy as np

import json

from census import Census
from us import states

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# sam's API key: 23c580dcbb047d99e93502f60681fc6430f2fc9a
c = Census("23c580dcbb047d99e93502f60681fc6430f2fc9a", year = 2020)


### Population Density

In [14]:
# https://www.census.gov/data/tables/time-series/dec/density-data-text.html
# renamed from "apportionment.csv"
pop_density = pd.read_csv("population_density.csv")
pop_density = pop_density[['Name', 'Geography Type', 'Year', 'Resident Population Density']]
pop_density = pop_density[(pop_density['Year'] == 2020) & (pop_density['Geography Type'] == "State")]
pop_density = pop_density[['Name', 'Resident Population Density']].reset_index(drop = True)

# 52 datapoints because DC and Puerto Rico count as states
pop_density.head()

,Name,Resident Population Density
0,Alabama,99.2
1,Alaska,1.3
2,Arizona,62.9
3,Arkansas,57.9
4,California,253.7


### Median Age

In [15]:
median_age_dictionary = c.acs5.state(('NAME', 'B01002_001E'), Census.ALL)
median_age = pd.DataFrame.from_dict(median_age_dictionary).rename(columns = {'NAME' : 'Name', 'B01002_001E' : 'Median Age'})
median_age = median_age[['Name', 'Median Age']].sort_values(by = ['Name']).reset_index(drop = True)
median_age


,Name,Median Age
0,Alabama,39.2
1,Alaska,34.6
2,Arizona,37.9
3,Arkansas,38.3
4,California,36.7
5,Colorado,36.9
6,Connecticut,41.1
7,Delaware,41.0
8,District of Columbia,34.1
9,Florida,42.2


### Median Income

In [16]:
median_income_dictionary = c.acs5.state(('NAME', 'B19013_001E'), Census.ALL)
median_income = pd.DataFrame.from_dict(median_income_dictionary).rename(columns = {'NAME' : 'Name', 'B19013_001E' : 'Median Household Income'})
median_income = median_income[['Name', 'Median Household Income']].sort_values(by = ['Name']).reset_index(drop = True)
median_income

,Name,Median Household Income
0,Alabama,52035.0
1,Alaska,77790.0
2,Arizona,61529.0
3,Arkansas,49475.0
4,California,78672.0
5,Colorado,75231.0
6,Connecticut,79855.0
7,Delaware,69110.0
8,District of Columbia,90842.0
9,Florida,57703.0


## Hypothesis Testing

*Likely just going to focus on either the 2020 presidential election because the FEC data for Trump's 2024 campaign is unclean, scattered, and not even on the main page as it was in previous elections.*

Electoral college and popular vote numbers by state: https://www.fec.gov/resources/cms-content/documents/2024presgeresults.pdf
- how does median age correlate with funding/spending
- how does population density correlate spending / voter turnout
- how does median income correlate with chosen candidate

### Median Age and Spending
*How does median age correlate with campaign spending?* \
*Do campaigns spend more on states/areas with a lower median age?*

### Population Density and Voter Turnout
*How does population density correlate with voter turnout?* \
*Do denser areas have more engaged voters?*

In [17]:
# the graph could be a dot plot where the x-axis is population density and the y-axis is voter turnout as a percentage of voting population
# (or maybe just percentage of overall population to make things easier)
# with an autogenerated line of best fit
# and then have the size of each dot scale with population size

### Median Income and Funding/Spending
*Do wealthier states contribute disproportionately more to campaigns?*
*Do poorer states receive more campaign spending?*

In [18]:
# if we choose to compare all three, it could be x-axis income, y-axis spending, and dot color candidate

### Campaign Spending and Chosen Candidate
*Does increased spending always lead to more votes?*